<a href="https://colab.research.google.com/github/JoanVale22/lab-simulation/blob/main/The_Rogue_Reviewer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import ipywidgets as widgets
from IPython.display import display, HTML
import webbrowser

# Adding a background sketch to the UI
display(HTML("""
    <style>
        body {
            background: url('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD...') no-repeat center center fixed;
            background-size: cover;
            font-family: Arial, sans-serif;
            color: white;
        }
        .content {
            background: rgba(0, 0, 0, 0.7);
            padding: 20px;
            border-radius: 10px;
            max-width: 600px;
            margin: 50px auto;
            text-align: center;
        }
        .content h1 {
            color: crimson;
        }
        .content button {
            margin: 10px 0;
        }
    </style>
    <div class='content'>
        <h1>The Rogue Reviewer</h1>
        <h2>A Data Poisoning Attack Adventure</h2>
        <p>Can you detect and prevent data poisoning before it's too late?</p>
        <hr>
    </div>
"""))

# Simulated IMDB dataset
data = {
    "review": [
        "Amazing movie with great acting!", "Horrible film, worst ever!",
        "Loved it, would watch again!", "Terrible experience, regret watching!",
        "A masterpiece, highly recommended!", "Awful script, waste of time!"
    ],
    "sentiment": [1, 0, 1, 0, 1, 0]
}
df = pd.DataFrame(data)

# Split data
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=10)
y_train = np.array(train_data["sentiment"])
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=10)
y_test = np.array(test_data["sentiment"])

# Function to inject poisoned data
def poison_data():
    global train_data
    poisoned_reviews = random.sample(range(len(train_data)), k=1)  # Inject one poisoned entry
    train_data.iloc[poisoned_reviews, 1] = 1 - train_data.iloc[poisoned_reviews, 1]  # Flip sentiment
    output_text.value = "⚠️ The dataset has been poisoned! Watch out!"

# Function to train the model
def train_model():
    model = Sequential([
        Embedding(input_dim=5000, output_dim=32, input_length=10),
        LSTM(32, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=2, verbose=1)

    predictions = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, predictions)
    model.save("bi_lstm_model.h5")  # Save the model
    output_text.value = f"✅ LSTM Model trained successfully! Accuracy: {accuracy:.2f}"

# Function to open Google Colab
def open_colab():
    colab_iframe.value = "<a href='https://colab.research.google.com/' target='_blank'>🚀 Open Google Colab</a>"

# Dialogue instructions
dialogue = widgets.HTML(value="""
    <div style="background-color: rgba(0, 0, 0, 0.8); padding: 15px; border-radius: 10px; color: white;">
        <p><b>Welcome Detective!</b></p>
        <p>Follow the steps to uncover the mystery:</p>
        <ul>
            <li>Step 1: Inject poison to simulate an attack.</li>
            <li>Step 2: Train your Bi-LSTM model and assess its performance after injected the poison.</li>
            <li>Step 3: Implement countermeasure to make a defense for the attack to to thwart future sabotage attempts.</li>
            <li>Step 4: Open the Google Colaboratory for the process of implemeting countermeasures.</li>
        </ul>
    </div>
""")

# Widgets for interaction
btn_poison = widgets.Button(
    description="💀 Inject Poisoned Data",
    button_style='danger',
    tooltip='Modify dataset by injecting poisoned data'
)
btn_poison.on_click(lambda _: poison_data())

btn_train = widgets.Button(
    description="🚀 Train Bi-LSTM Model",
    button_style='success',
    tooltip='Train the sentiment analysis model'
)
btn_train.on_click(lambda _: train_model())

btn_colab = widgets.Button(
    description="🌐 Open Google Colab",
    button_style='info',
    tooltip='Open Google Colab in a popup'
)
btn_colab.on_click(lambda _: open_colab())

output_text = widgets.HTML(value="")
colab_iframe = widgets.HTML(value="")

# Layout without the "Next" button
display(widgets.VBox([
    dialogue,
    widgets.HBox([btn_poison, btn_train, btn_colab]),
    output_text,
    colab_iframe
]))
